## Import Packages

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier

# Problem Understanding

The Vehicle Safety Board of Chicago has determine that it is necessary to know and understand the primary contributory cause of a car accident. Therefore, it will be possible to recognise patterns and consequently "it could create guidelines to implement preventive actions."
Based on the data, we are going to predict if the cause of a car accident if influence by the weather, location, time or by the number of passengers.
From the created models, we want to estimate the global/generic/general/average cost of having a injury person or the cost a average crash.


Predict the frequency of traffic accidents between the road geometric variables, traffic characteristics, and environmental factors.


# Data Understanding



In [2]:
# Crashes Dataset
crash = pd.read_csv('Traffic_Crashes_-_Crashes.csv')
# crash.info()

In [3]:
# People Dataset
people = pd.read_csv('Traffic_Crashes_-_People.csv')
# people.head()

/Users/Vittorio/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (12,13,20,21,22,23,24,25,26,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Vehicles Dataset
vehicles = pd.read_csv('Traffic_Crashes_-_Vehicles.csv')
# crashes.head()

/Users/Vittorio/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,55,58,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Selecting Features

In [5]:
# Selecting specific features on crashes dataset
crashes = crash[['CRASH_RECORD_ID', 'POSTED_SPEED_LIMIT', 'WEATHER_CONDITION', 
                 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'ROADWAY_SURFACE_COND',
                 'ROAD_DEFECT', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE',
                 'BEAT_OF_OCCURRENCE','CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH']]

In [6]:
# Selecting features on people's dataset
peoples = people[['CRASH_RECORD_ID','PERSON_TYPE', 'SEX', 'AGE']]

In [7]:
# Selecting features on vehicle's dataset
vehicle = vehicles[['CRASH_RECORD_ID', 'VEHICLE_TYPE', 'MANEUVER']]

## Combining Datasets

In [8]:
#  Merging crashes with people datastes
df_cp = pd.merge(crashes, peoples, on = 'CRASH_RECORD_ID', how = 'left')

In [9]:
# Merging df_cp with vehicles dataset
df_x = pd.merge(df_cp, vehicle, on = 'CRASH_RECORD_ID', how = 'left')

In [10]:
# Selecting only class Driver
df_t = df_x[(df_x.PERSON_TYPE == 'DRIVER') & (df_x.AGE >=16) & (df_x.AGE <= 100)]
# df_t.isnull().sum()

In [12]:
dfq = df_t.drop_duplicates(subset = 'CRASH_RECORD_ID', keep='first')
dfq.isnull().sum()

CRASH_RECORD_ID               0
POSTED_SPEED_LIMIT            0
WEATHER_CONDITION             0
LIGHTING_CONDITION            0
FIRST_CRASH_TYPE              0
ROADWAY_SURFACE_COND          0
ROAD_DEFECT                   0
PRIM_CONTRIBUTORY_CAUSE       0
SEC_CONTRIBUTORY_CAUSE        0
BEAT_OF_OCCURRENCE            4
CRASH_HOUR                    0
CRASH_DAY_OF_WEEK             0
CRASH_MONTH                   0
PERSON_TYPE                   0
SEX                           0
AGE                           0
VEHICLE_TYPE               2711
MANEUVER                   2711
dtype: int64

In [23]:
# Dropping all nun values
df = dfq.dropna()
df.reset_index(inplace=True)

In [24]:
# Creating dummy variables 
x_feats = ['WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
           'ROADWAY_SURFACE_COND','ROAD_DEFECT','VEHICLE_TYPE', 'MANEUVER',
           'SEX', 'SEC_CONTRIBUTORY_CAUSE','PERSON_TYPE']
    
tre = pd.get_dummies(df[x_feats], drop_first=True, dtype=float)
tre

,WEATHER_CONDITION_BLOWING SNOW,WEATHER_CONDITION_CLEAR,WEATHER_CONDITION_CLOUDY/OVERCAST,WEATHER_CONDITION_FOG/SMOKE/HAZE,WEATHER_CONDITION_FREEZING RAIN/DRIZZLE,WEATHER_CONDITION_OTHER,WEATHER_CONDITION_RAIN,WEATHER_CONDITION_SEVERE CROSS WIND GATE,WEATHER_CONDITION_SLEET/HAIL,WEATHER_CONDITION_SNOW,...,SEC_CONTRIBUTORY_CAUSE_PHYSICAL CONDITION OF DRIVER,SEC_CONTRIBUTORY_CAUSE_RELATED TO BUS STOP,SEC_CONTRIBUTORY_CAUSE_ROAD CONSTRUCTION/MAINTENANCE,SEC_CONTRIBUTORY_CAUSE_ROAD ENGINEERING/SURFACE/MARKING DEFECTS,SEC_CONTRIBUTORY_CAUSE_TEXTING,SEC_CONTRIBUTORY_CAUSE_TURNING RIGHT ON RED,SEC_CONTRIBUTORY_CAUSE_UNABLE TO DETERMINE,SEC_CONTRIBUTORY_CAUSE_UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED),"SEC_CONTRIBUTORY_CAUSE_VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)",SEC_CONTRIBUTORY_CAUSE_WEATHER
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317352,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
317353,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
317354,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
317355,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
y_dummy = ['PRIM_CONTRIBUTORY_CAUSE']
y_dum = pd.get_dummies(df[y_dummy], drop_first=True, dtype=float)
y_dum

,PRIM_CONTRIBUTORY_CAUSE_BICYCLE ADVANCING LEGALLY ON RED LIGHT,PRIM_CONTRIBUTORY_CAUSE_CELL PHONE USE OTHER THAN TEXTING,PRIM_CONTRIBUTORY_CAUSE_DISREGARDING OTHER TRAFFIC SIGNS,PRIM_CONTRIBUTORY_CAUSE_DISREGARDING ROAD MARKINGS,PRIM_CONTRIBUTORY_CAUSE_DISREGARDING STOP SIGN,PRIM_CONTRIBUTORY_CAUSE_DISREGARDING TRAFFIC SIGNALS,PRIM_CONTRIBUTORY_CAUSE_DISREGARDING YIELD SIGN,PRIM_CONTRIBUTORY_CAUSE_DISTRACTION - FROM INSIDE VEHICLE,PRIM_CONTRIBUTORY_CAUSE_DISTRACTION - FROM OUTSIDE VEHICLE,"PRIM_CONTRIBUTORY_CAUSE_DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)",...,PRIM_CONTRIBUTORY_CAUSE_PHYSICAL CONDITION OF DRIVER,PRIM_CONTRIBUTORY_CAUSE_RELATED TO BUS STOP,PRIM_CONTRIBUTORY_CAUSE_ROAD CONSTRUCTION/MAINTENANCE,PRIM_CONTRIBUTORY_CAUSE_ROAD ENGINEERING/SURFACE/MARKING DEFECTS,PRIM_CONTRIBUTORY_CAUSE_TEXTING,PRIM_CONTRIBUTORY_CAUSE_TURNING RIGHT ON RED,PRIM_CONTRIBUTORY_CAUSE_UNABLE TO DETERMINE,PRIM_CONTRIBUTORY_CAUSE_UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED),"PRIM_CONTRIBUTORY_CAUSE_VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)",PRIM_CONTRIBUTORY_CAUSE_WEATHER
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
317353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
317354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
317355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
qw = df[['BEAT_OF_OCCURRENCE', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK']]

## Imbalanced Classes


# Data Preparation

## Split your X data in train and test datasets¶


In [113]:
X = pd.concat([qw, tre], axis=1)
y = y_dum['PRIM_CONTRIBUTORY_CAUSE_FOLLOWING TOO CLOSELY']

In [114]:
# Split X data in train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split train data in train and validation datasets
X_train_s, X_val, y_train_s, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(len(X_train_s), len(X_val), len(y_train_s), len(y_val))

203108 50777 203108 50777


In [115]:
print('X_train_s:', X_train_s.shape)
print('y_train_s:', y_train_s.shape)
print('X_val:', X_val.shape)
print('y_val:', y_val.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

X_train_s: (203108, 137)
y_train_s: (203108,)
X_val: (50777, 137)
y_val: (50777,)
X_test: (63472, 137)
y_test: (63472,)


## Scale the 3 datasets using StandardScaler¶


In [116]:
scaler = StandardScaler()
scaler.fit(X_train_s)

X_train_scaled = scaler.transform(X_train_s)
X_val_scaled = scaler.transform(X_val) 
X_test = scaler.transform(X_test)

# Random Forest Classifier

In [117]:
# Instatiating our folds so that we can measure performance on the same basis
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3,
                      random_state=777,
                      shuffle=True)

In [118]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=50,
                             random_state=777)

In [121]:
from sklearn.model_selection import GridSearchCV

rfc_param_grid = {'max_depth':range(1,11,100),'min_samples_leaf':[5,10,15]}

opt_rfc = GridSearchCV(rfc, param_grid, cv=skf, scoring='roc_auc', n_jobs=-1)
opt_rfc.fit(X_train_scaled, y_train_s)
best_rfc = opt_rfc.best_estimator_

opt_rfc.best_params_

{'max_depth': 8, 'min_samples_leaf': 10}

In [124]:
opt_rfc.best_score_

0.8940709596371833

In [125]:
rf_predi = opt_rfc.predict(X_val)
rf_predI_proba = opt_rfc.predict_proba(X_val)[:,1]

In [126]:
print(accuracy_score(y_val, rf_predi))
print(roc_auc_score(y_val, rf_predI_proba))

0.869271520570337
0.8709720164818554
